In [1]:
import requests
import json
import pandas as pd
from steam import Steam
from tqdm import tqdm
import csv
import time

In [2]:
KEY = '0BAEAA44F5C3C9967C53E41AE9B50F25'
steam = Steam(KEY)

In [3]:
URL = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
r = requests.get(url=URL)

In [4]:
apps_data = r.json()
apps_data = apps_data['applist']['apps']

In [5]:
with open('output_files/apps.json', 'w') as F:
    json.dump(apps_data, F)

In [6]:
df = pd.read_json('output_files/apps.json')
df.to_csv('output_files/apps.csv', index=None)

In [7]:
app_ids = df['appid']

In [10]:
with open('output_files/games.csv', 'w') as F:
    writer = csv.writer(F)
    sample = steam.apps.get_app_details(2281650)
    sample = json.loads(sample)
    cols = ['name', 'steam_appid', 'detailed_description', 'about_the_game', 'short_description', 'total_recommendations', ]
    writer.writerow(cols)
    for i, row in tqdm(app_ids.to_frame().iterrows(), total = app_ids.shape[0]):
        sample = steam.apps.get_app_details(row['appid'])
        sample = json.loads(sample)
        if sample[str(row['appid'])]['success'] == True: 
            game_data = sample[str(row['appid'])]['data']
            if game_data['type'] == 'game':
                data = []
                data.append(game_data['name'])
                data.append(game_data['steam_appid'])
                data.append(game_data['detailed_description'])
                data.append(game_data['about_the_game'])
                data.append(game_data['short_description'])
                try:
                    data.append(game_data['recommendations']['total'])
                except KeyError:
                    data.append(0)
                writer.writerow(data)
        if i%120 == 0:
            time.sleep(180)

  0%|          | 231/163699 [05:16<62:14:13,  1.37s/it]  


TypeError: 'NoneType' object is not subscriptable